In [65]:
from pathlib import Path
import pandas as pd, requests

ROOT = Path().resolve()
DATA_RAW  = ROOT.parent / "data" / "raw"
DATA_PROC = ROOT.parent / "data" / "processed"
DATA_RAW.mkdir(parents=True, exist_ok=True)
DATA_PROC.mkdir(parents=True, exist_ok=True)

In [66]:
#descarga

BASE = "https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data"

DATASETS = {
    # Código (Eurostat) : alias
    "lfsa_ehomp":    "telework",          # Teletrabajo (% empleo)
    "lfso_21jsat04": "jobsat_wfh_flex",   # Job satisfaction × WFH × flexibilidad
    "lfso_21jsat01": "jobsat_baseline",   # Job satisfaction (baseline)
}

def eurostat_url(dataset: str, fmt="SDMX-CSV"):
    # Filtramos años 2015..2024 para limitar tamaño de respuesta
    return (
        f"{BASE}/{dataset}"
        f"?time=2015:2024"
        f"&format={fmt}"
        f"&compressed=false"
        f"&lang=en"
    )

def download_dataset(dataset: str):
    # 1º intento: SDMX-CSV
    url_csv = eurostat_url(dataset, fmt="SDMX-CSV")
    r = requests.get(url_csv, timeout=120)
    if r.ok and r.content.strip():
        out = DATA_RAW / f"{dataset}.csv"
        out.write_bytes(r.content)
        return out

    # Fallback: TSV
    url_tsv = eurostat_url(dataset, fmt="TSV")
    r2 = requests.get(url_tsv, timeout=120)
    r2.raise_for_status()
    out = DATA_RAW / f"{dataset}.tsv"
    out.write_bytes(r2.content)
    return out

raw_files = {}
for code, _alias in DATASETS.items():
    path = download_dataset(code)
    raw_files[code] = path
    print(code, "→ guardado en", path)


lfsa_ehomp → guardado en C:\Users\mdmg9\OneDrive\Vac UE\Sep25\eu-lfs-telework-2015-2024\data\raw\lfsa_ehomp.csv
lfso_21jsat04 → guardado en C:\Users\mdmg9\OneDrive\Vac UE\Sep25\eu-lfs-telework-2015-2024\data\raw\lfso_21jsat04.csv
lfso_21jsat01 → guardado en C:\Users\mdmg9\OneDrive\Vac UE\Sep25\eu-lfs-telework-2015-2024\data\raw\lfso_21jsat01.csv


In [67]:
#limpieza lfsa_ehomp.csv

import pandas as pd
from pathlib import Path

RAW = Path("../data/raw/lfsa_ehomp.csv")   # o .tsv
OUT = Path("../data/processed/lfsa_ehomp_2015_2024_clean.csv")

df = pd.read_csv(RAW)
df = df.rename(columns={"TIME_PERIOD":"time","OBS_VALUE":"value"})
df["value"] = pd.to_numeric(df["value"], errors="coerce")

# Años 2015–2024
df = df[df["time"].astype(str).str.fullmatch(r"\d{4}")]
df["time"] = df["time"].astype(int)
df = df[df["time"].between(2015, 2024)]
df = df[df["value"].notna()]

# Normaliza EU28 -> EU27_2020 si aparece
if "geo" in df.columns:
    df["geo"] = df["geo"].replace({"EU28":"EU27_2020"})

# --- Selección automática de "totales" ---

# SEX: esperamos 'T'
if "sex" in df.columns:
    before = len(df)
    df = df[df["sex"].isin(["T","TOTAL","All sexes"])]
    print(f"Filtrado sex: {before} -> {len(df)} filas (mantengo T/TOTAL)")

# AGE: prioridad Y20-64 > Y15-64 > Y15-74 > TOTAL
age_priority = ["Y20-64", "Y15-64", "Y15-74", "TOTAL", "All ages"]
if "age" in df.columns:
    available = [a for a in age_priority if a in set(df["age"])]
    if available:
        pick = available[0]
        before = len(df)
        df = df[df["age"] == pick]
        print(f"Filtrado age: escojo {pick}  ({before} -> {len(df)} filas)")
    else:
        print("Aviso: no encuentro ninguna de las edades prioridad; continúo sin filtrar age.")

# WSTATUS: prioridad EMP > TOTAL; si nada de eso, agruparé promediando
wstatus_pick = None
if "wstatus" in df.columns:
    uw = set(df["wstatus"])
    if "EMP" in uw:
        wstatus_pick = "EMP"
    elif "TOTAL" in uw:
        wstatus_pick = "TOTAL"
    if wstatus_pick is not None:
        before = len(df)
        df = df[df["wstatus"] == wstatus_pick]
        print(f"Filtrado wstatus: escojo {wstatus_pick}  ({before} -> {len(df)} filas)")
    else:
        print("Aviso: no hay EMP/TOTAL en wstatus; luego promediaré por wstatus (aprox.).")

# Frecuencia de teletrabajo: USU/SMT
freqcat_keep = {"USU","SMT"}
if "frequenc" in df.columns:
    before = len(df)
    df = df[df["frequenc"].isin(freqcat_keep)]
    print(f"Filtrado frequenc (USU/SMT): {before} -> {len(df)} filas")
else:
    raise ValueError("No encuentro columna 'frequenc' con categorías USU/SMT.")

# --- Construcción de telework_any ---

group_cols = ["geo","time"]
# Si no pudimos fijar wstatus (no había EMP/TOTAL), promediamos entre wstatus (aproximación)
if "wstatus" in df.columns and wstatus_pick is None:
    group_cols.append("wstatus")

wide = df.pivot_table(index=group_cols, columns="frequenc", values="value", aggfunc="mean")
for c in ("USU","SMT"):
    if c not in wide.columns:
        wide[c] = 0.0

wide["telework_any"] = wide["USU"].fillna(0) + wide["SMT"].fillna(0)
wide = wide.reset_index()

# Si aún hay wstatus (porque promediamos), colapsa a país-año (media simple, aprox.)
if "wstatus" in wide.columns and wstatus_pick is None:
    before = len(wide)
    wide = wide.groupby(["geo","time"], as_index=False)[["USU","SMT","telework_any"]].mean()
    print(f"Agrupado por wstatus (aprox.): {before} -> {len(wide)} filas")

wide = wide.sort_values(["geo","time"]).reset_index(drop=True)

# Guardar
OUT.parent.mkdir(parents=True, exist_ok=True)
wide.to_csv(OUT, index=False)
print("Guardado:", OUT)
display(wide.head())





Filtrado sex: 479727 -> 169075 filas (mantengo T/TOTAL)
Filtrado age: escojo Y20-64  (169075 -> 7999 filas)
Filtrado wstatus: escojo EMP  (7999 -> 1068 filas)
Filtrado frequenc (USU/SMT): 1068 -> 712 filas
Guardado: ..\data\processed\lfsa_ehomp_2015_2024_clean.csv


frequenc,geo,time,SMT,USU,telework_any
0,AT,2015,12.2,10.5,22.7
1,AT,2016,12.4,10.2,22.6
2,AT,2017,12.5,9.7,22.2
3,AT,2018,12.0,10.2,22.2
4,AT,2019,12.4,10.2,22.6


In [68]:
#Limpieza lfso_21jsat04 (satisfacción × teletrabajo × flexibilidad) 
import pandas as pd
from pathlib import Path
import numpy as np
import re

RAW = Path("../data/raw/lfso_21jsat01.csv")   # o .tsv
OUT = Path("../data/processed/lfso_21jsat01_2015_2024_clean.csv")

df0 = pd.read_csv(RAW)

# Normaliza nombres clave
df = df0.rename(columns={"TIME_PERIOD":"time", "OBS_VALUE":"value"}).copy()

# Extrae año YYYY aunque venga como '2021-01'/'2021Q1'
if "time" in df.columns:
    df["time_year"] = df["time"].astype(str).str.extract(r"(\d{4})")
    df["time_year"] = pd.to_numeric(df["time_year"], errors="coerce")
    # nos quedamos con 2015..2024 si existen (si solo hay 2021, quedará 2021)
    df = df[df["time_year"].between(2015, 2024, inclusive="both")]
    df["time"] = df["time_year"].astype("Int64")
    df.drop(columns=["time_year"], inplace=True)
else:
    raise ValueError("No encuentro columna temporal (TIME_PERIOD/time).")

# A numérico; NO filtramos por unit/freq para no vaciar
df["value"] = pd.to_numeric(df["value"], errors="coerce")

# Quita NA y baja fiabilidad si quieres ser estricto
df = df[df["value"].notna()]
if "OBS_FLAG" in df.columns:
    df = df[~df["OBS_FLAG"].fillna("").str.contains("u")]

# Totales razonables de sexo/edad (si existen)
if "sex" in df.columns:
    df = df[df["sex"].isin(["T","TOTAL","All sexes"]) | df["sex"].isna()]
if "age" in df.columns:
    df = df[df["age"].isin(["Y20-64","Y15-64","Y15-74","TOTAL","All ages"]) | df["age"].isna()]

# Algunos ficheros traen 'unit' = THS_PER (miles); otros 'PC' (porcentaje).
# Para poder comparar países, construimos proporción dentro de cada grupo.
# Grupo de referencia (incluye c_birth/isced11 si existen):
group_keys = ["geo","time"]
for extra in ["c_birth","isced11"]:
    if extra in df.columns:
        group_keys.append(extra)

# Necesitamos 'lev_satis' (si no existe, no podemos construir HIGH):
if "lev_satis" not in df.columns:
    # fallback: usa 'value' promedio como proxy
    dfm = df.groupby(["geo","time"], as_index=False)["value"].mean()
    dfm = dfm.rename(columns={"value":"share_high"})
else:
    # Suma por grupo (para proporciones si unit=THS_PER) o normaliza si unit ya es PC
    gsum = df.groupby(group_keys, as_index=False)["value"].sum().rename(columns={"value":"group_total"})
    dff = df.merge(gsum, on=group_keys, how="left")

    # Si 'unit' es PC, la suma de categorías suele ~100, si es THS_PER suma personas.
    # Construimos share = value / group_total, pero si group_total ~0 o ~100,
    # este cociente también sirve (quedará ~ value/100 si 'PC').
    dff["share"] = np.where(dff["group_total"]>0, dff["value"]/dff["group_total"], np.nan)

    # Nos quedamos con HIGH
    df_high = dff[dff["lev_satis"].astype(str).str.upper().eq("HIGH")].copy()
    dfm = df_high.groupby(["geo","time"], as_index=False)["share"].mean().rename(columns={"share":"share_high"})

# Orden y guardado
dfm = dfm.sort_values(["geo","time"]).reset_index(drop=True)
OUT.parent.mkdir(parents=True, exist_ok=True)
dfm.to_csv(OUT, index=False)
print("Guardado:", OUT, "| filas:", len(dfm))
display(dfm.head())

# Diagnóstico útil:
print("\nAños presentes:", sorted(dfm["time"].unique()))
print("Países (muestra):", sorted(dfm["geo"].unique())[:10], "...")


Guardado: ..\data\processed\lfso_21jsat01_2015_2024_clean.csv | filas: 31


,geo,time,share_high
0,AT,2021,0.089542
1,BE,2021,0.085934
2,BG,2021,0.048013
3,CH,2021,0.104222
4,CY,2021,0.103489



Años presentes: [2021]
Países (muestra): ['AT', 'BE', 'BG', 'CH', 'CY', 'CZ', 'DE', 'DK', 'EA20', 'EE'] ...
